In [1]:
import ee
# ee.Authenticate()
try:
  ee.Initialize()
  print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
  print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Google Earth Engine has initialized successfully!


In [ ]:
srtm = ee.Image("USGS/SRTMGL1_003")
# Total now: 6000
num = 3000
# region = ee.Geometry.Polygon([[143.78179968403583,-37.66494606551627],
#                               [149.14312780903583,-36.26048144242306],
#                               [152.46046847751526,-29.591409579420986],
#                               [149.75836218403583,-22.002437813303377],
#                               [131.21344030903583,-14.113450343617195],
#                               [126.71735793193226,-27.064383319090037],
#                               [143.78179968403583,-37.66494606551627]])
# region = ee.FeatureCollection('users/thiennust274/mekongbasin')
region = ee.FeatureCollection('users/thiennust274/aus_mountainous')
region = region.geometry()
FOLDER = 'aus_mountainous_add'

rand_points = ee.FeatureCollection.randomPoints(region, num)
rand_regions = rand_points.map(lambda p: p.buffer(5000).bounds())

images = rand_regions.map(lambda region: srtm.clip(region))
list_images = images.toList(images.size().getInfo())
list_regions = rand_regions.toList(rand_regions.size().getInfo())
for i in range(num):
    img = ee.Image(list_images.get(i))
    region = ee.Feature(list_regions.get(i))
    name = 'STRM' + str(i+ 6000)
    taskcfg = {
        'description': name,
        'image': img,
        'region': region.geometry(),
        'scale': 30,
        'folder': FOLDER,
        'fileNamePrefix': name,
    }
    ee.batch.Export.image.toDrive(**taskcfg).start()